<a href="https://colab.research.google.com/github/Mithesh6369894902/CODSOFT-INTERNSHIP/blob/main/RECOMMENDATION_SYSTEM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Importing the Necessities**

In [1]:
!pip install annoy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 647.5/647.5 kB 5.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for annoy: filename=annoy-1.17.3-cp310-cp310-linux_x86_64.whl size=550739 sha256=9921c30d05fccf6eadcab4b1af5e583dcde21a9eec6d5661360949903444b4d5
  Stored in directory: /root/.cache/pip/wheels/64/8a/da/f714bcf46c5efdcfcac0559e63370c21abe961c48e3992465a
Successfully built annoy


In [2]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from annoy import AnnoyIndex

**Data Collection and Pre-processing**

In [3]:
# Define data types for each column
dtype = {
    'danceability': float,
    'energy': float,
    'key': int,
    'loudness': float,
    'mode': int,
    'speechiness': float,
    'acousticness': float,
    'instrumentalness': float,
    'liveness': float,
    'valence': float,
    'tempo': float,
    'type': str,
    'id': str,
    'uri': str,
    'track_href': str,
    'analysis_url': str,
    'duration_ms': int,
    'time_signature': int,
    'genre': str,
    'song_name': str,
    'title': str
}

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
# Read the CSV file with the specified data types
data = pd.read_csv('/content/drive/MyDrive/Spotify.csv', dtype=dtype, low_memory=False)

# Select features for similarity calculation
features = ['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness',
            'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo']

# Normalize the features
scaler = StandardScaler()
data[features] = scaler.fit_transform(data[features])

**Building Annoy Index with 10 Trees**

In [6]:
# Initialize Annoy index
f = len(features)
t = AnnoyIndex(f, 'angular')

# Add items to the Annoy index
for i, row in data.iterrows():
    t.add_item(i, row[features])

# Build the Annoy index
t.build(10)  # 10 trees

True

**Function to Recommend Songs**

In [7]:
# Function to recommend songs
def recommend_songs(song_name, data, t, top_n=3):
    if song_name not in data['song_name'].values:
        raise ValueError(f"Song '{song_name}' not found in the dataset.")

    song_index = data.index[data['song_name'] == song_name].tolist()[0]
    similar_items = t.get_nns_by_item(song_index, top_n + 1)  # +1 to include the song itself
    similar_items.remove(song_index)  # Remove the song itself

    # Return the recommended song names
    return data.iloc[similar_items]['song_name'].tolist()

**Example Recommendation**

In [8]:
# Example: Recommend songs similar to a specific song
try:
    recommendations = recommend_songs('Pathology', data, t)
    print(f"Recommended songs similar to 'Pathology': {recommendations}")
except ValueError as e:
    print(e)

Recommended songs similar to 'Pathology': ['Can Of Worms', 'Can Of Worms', 'Can Of Worms']
